In [10]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from data_extraction.dummy_data_extractor import extract_dummy_data

In [18]:
master_table = extract_dummy_data("dummy_data")

C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


In [120]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from data_extraction.dummy_data_extractor import extract_dummy_data

master_table = extract_dummy_data("dummy_data")

# Extract the relevant dataframe from 'master_table'
df = master_table.at[1, "his"]

df.reset_index(inplace=True)
df = df.dropna()

# Keep only the first two columns
df = df.iloc[:, :2]

# renaming columns
df.columns = ['ds', 'temp']

# Remove ' Dubai' from the datetime strings
df['ds'] = df['ds'].str.replace(' Dubai', '', regex=False)

# Convert the 'ds' column to datetime format
df['ds'] = pd.to_datetime(df['ds'], format="%Y-%m-%dT%H:%M:%S%z")


C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


In [122]:
# Drop rows where datetime parsing failed
df = df.dropna(subset=['ds'])

In [124]:
# Clean temperature column and convert to numeric
df['temp'] = df['temp'].str.replace('°C', '').astype(float)


In [126]:
# Rename columns for convenience
df.columns = ['ds', 'y']


In [128]:
# Separate data for temperature
df_temp = df.copy()

# Ensure 'ds' column is timezone-naive
df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)

In [130]:
# Initialize Prophet models with tuned hyperparameters
model_temp = Prophet(seasonality_mode='additive',     # Adjust based on data exploration
                     interval_width=0.95,              # Adjust prediction interval if needed
                     changepoint_prior_scale=0.01)    # Tune based on data patterns

In [132]:
# Fit the models
model_temp.fit(df_temp)

12:51:57 - cmdstanpy - INFO - Chain [1] start processing
12:51:57 - cmdstanpy - INFO - Chain [1] done processing


In [134]:
length_of_missing_data = pd.Timedelta('0 days 23:30:00')

In [110]:
length_of_missing_data

Timedelta('0 days 23:30:00')

In [136]:

data_logging_interval = pd.Timedelta('0 days 00:05:00')


In [138]:
 # number of predictions
samples = int(length_of_missing_data/data_logging_interval) + 1

In [108]:
samples

283

In [140]:
# Create future DataFrames for both temp and new_point (next 200 samples, assuming 5-minute intervals)
future_temp = model_temp.make_future_dataframe(periods=samples, freq='5T')

In [142]:
# Predict the future values
forecast_temp = model_temp.predict(future_temp)

In [144]:
forecast_temp

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-14 21:10:00,24.367781,23.883961,25.342836,24.367781,24.367781,0.264977,0.264977,0.264977,0.264977,0.264977,0.264977,0.0,0.0,0.0,24.632758
1,2023-03-14 21:15:00,24.368370,23.924441,25.364407,24.368370,24.368370,0.241976,0.241976,0.241976,0.241976,0.241976,0.241976,0.0,0.0,0.0,24.610346
2,2023-03-14 21:20:00,24.368959,23.856130,25.376344,24.368959,24.368959,0.218976,0.218976,0.218976,0.218976,0.218976,0.218976,0.0,0.0,0.0,24.587935
3,2023-03-14 21:25:00,24.369548,23.812498,25.270631,24.369548,24.369548,0.196038,0.196038,0.196038,0.196038,0.196038,0.196038,0.0,0.0,0.0,24.565586
4,2023-03-14 21:30:00,24.370137,23.814294,25.274879,24.370137,24.370137,0.173219,0.173219,0.173219,0.173219,0.173219,0.173219,0.0,0.0,0.0,24.543357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,2023-03-20 00:20:00,22.253753,21.036540,22.826916,21.677905,22.751828,-0.315375,-0.315375,-0.315375,-0.315375,-0.315375,-0.315375,0.0,0.0,0.0,21.938378
1476,2023-03-20 00:25:00,22.251469,21.078549,22.822481,21.669657,22.752692,-0.322072,-0.322072,-0.322072,-0.322072,-0.322072,-0.322072,0.0,0.0,0.0,21.929398
1477,2023-03-20 00:30:00,22.249185,21.040391,22.794423,21.661408,22.753320,-0.328717,-0.328717,-0.328717,-0.328717,-0.328717,-0.328717,0.0,0.0,0.0,21.920469
1478,2023-03-20 00:35:00,22.246901,21.055972,22.771148,21.654381,22.753235,-0.335356,-0.335356,-0.335356,-0.335356,-0.335356,-0.335356,0.0,0.0,0.0,21.911545


In [1]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from data_extraction.dummy_data_extractor import extract_dummy_data

# Assuming 'extract_dummy_data' function extracts data into 'master_table'
master_table = extract_dummy_data("dummy_data")

# Extract the relevant dataframe from 'master_table'
df = master_table.at[1, "his"]

# Reset index and drop NA values
df.reset_index(inplace=True)
df = df.dropna()

# Rename columns for consistency
df.columns = ['ds', 'temp', 'new_point']

# Remove ' Dubai' from the datetime strings
df['ds'] = df['ds'].str.replace(' Dubai', '', regex=False)

# Convert the 'ds' column to datetime format
df['ds'] = pd.to_datetime(df['ds'], format="%Y-%m-%dT%H:%M:%S%z")

# Extract numeric temperature values
df['temp'] = df['temp'].str.extract('(\d+\.\d+)').astype(float)

# Print the DataFrame to check
print(df)

# Split data into train and test sets
train_size = int(len(df) * 0.8)  # 80% for training
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

# Prepare the dataframe with required column names for Prophet
train_df_prophet = train_df[['ds', 'temp']].copy()
train_df_prophet.rename(columns={'temp': 'y'}, inplace=True)  # Rename 'temp' to 'y'

# Initialize Prophet model
model = Prophet()

# Fit the model with training data
model.fit(train_df_prophet)

# Make future predictions for the test set
future = model.make_future_dataframe(periods=len(test_df), freq='H')
forecast = model.predict(future)

# Calculate RMSE (Root Mean Squared Error)
y_true = test_df['temp'].values
y_pred = forecast['yhat'].values[-len(test_df):]
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"RMSE: {rmse}")

# Save test set to CSV
test_df.to_csv(r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\test.csv', index=False)


C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")
C:\Users\syounas\AppData\Local\Temp\ipykernel_19424\4232964390.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ds'] = df['ds'].str.replace(' Dubai', '', regex=False)
C:\Users\syounas\AppData\Local\Temp\ipykernel_19424\4232964390.py:24: SettingWithCopyWarning: 
A valu

                            ds       temp new_point
0    2023-03-14 21:10:00+04:00  24.269638    33.13%
1    2023-03-14 21:15:00+04:00  24.269638    33.13%
2    2023-03-14 21:20:00+04:00  24.269638    33.13%
3    2023-03-14 21:25:00+04:00  24.269638    33.13%
4    2023-03-14 21:30:00+04:00  24.269638    33.13%
...                        ...        ...       ...
1195 2023-03-19 00:45:00+04:00  22.633480    29.43%
1196 2023-03-19 00:50:00+04:00  22.633480    30.55%
1197 2023-03-19 00:55:00+04:00  22.633480    30.51%
1198 2023-03-19 01:00:00+04:00  22.633480    30.51%
1199 2023-03-19 01:05:00+04:00  22.633480    30.51%

[1197 rows x 3 columns]


ValueError: Column ds has timezone specified, which is not supported. Remove timezone.

In [2]:
from fbprophet import Prophet
import pandas as pd

def fProphet(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Inputs
    df: df used for training set (from SS)
    length_of_missing_data: interval length of missing data (from SS)
    data_logging_interval: data logging interval - called from the hisDQInterval tag on the point (from SS)

    Output
    forecasts_df: dataframe with predictions for the period missing data. Index names as ts, values column named as "v0"
    """

    # Rename the first column as "target"
    new_column_name = "target"
    df = df.rename(columns={df.columns[0]: new_column_name})

    # Keep only the history BEFORE the start of the data quality issue, since this is a statistical model not ML model
    df = df[df.index < dqStart]

    # Format the df to Prophet format
    df = df.reset_index()
    df = df.rename(columns={df.columns[0]: 'ds', df.columns[1]: "y"})

    # Number of predictions
    horizon = int(length_of_missing_data / data_logging_interval) + 1

    # Frequency (Prophet will infer this from the data, but can be specified)
    freq = f'{data_logging_interval.total_seconds() / 3600}H'

    # Initialize and fit the Prophet model
    model = Prophet()
    model.fit(df)

    # Make a dataframe to hold predictions
    future = model.make_future_dataframe(periods=horizon, freq=freq)

    # Make predictions
    forecast = model.predict(future)

    # Filter out the predictions corresponding to the missing data period
    forecasts_df = forecast[['ds', 'yhat']].tail(horizon)
    forecasts_df = forecasts_df.rename(columns={"ds": "timestamp", "yhat": "dynamicOptimizedTheta"})
    forecasts_df.set_index("timestamp", inplace=True)

    return forecasts_df

                                ds                  temp
0  2023-03-14T21:10:00+04:00 Dubai  24.269638061523438°C
1  2023-03-14T21:15:00+04:00 Dubai  24.269638061523438°C
2  2023-03-14T21:20:00+04:00 Dubai  24.269638061523438°C
3  2023-03-14T21:25:00+04:00 Dubai  24.269638061523438°C
4  2023-03-14T21:30:00+04:00 Dubai  24.269638061523438°C


C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


In [ ]:







import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the data
file_path = r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\Train_1.csv'
data = pd.read_csv(file_path)

# Rename columns for convenience
data.columns = ['ts', 'temp']

# Convert 'ts' column to datetime and handle timezone offset
data['ts'] = pd.to_datetime(data['ts'].str.replace(' Dubai', ''), errors='coerce')
# Drop rows where datetime parsing failed
data = data.dropna(subset=['ts'])

# Clean temperature column and convert to numeric
data['temp'] = data['temp'].str.replace('°C', '').astype(float)

# Separate data for temperature
df_temp = data[['ts', 'temp']].rename(columns={'ts': 'ds', 'temp': 'y'})

# Ensure 'ds' column is timezone-naive
df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)

# Initialize Prophet models with tuned hyperparameters
model_temp = Prophet(seasonality_mode='additive',     # Adjust based on data exploration
                     interval_width=0.95,              # Adjust prediction interval if needed
                     changepoint_prior_scale=0.01)    # Tune based on data patterns

# Fit the models
model_temp.fit(df_temp)

# Create future DataFrames for both temp and new_point (next 200 samples, assuming 5-minute intervals)
future_temp = model_temp.make_future_dataframe(periods=200, freq='5T')

# Predict the future values
forecast_temp = model_temp.predict(future_temp)

# Calculate RMSE for the last 200 points in the training data
forecast_train = model_temp.predict(df_temp)
y_true = df_temp['y'].values[-200:]
y_pred = forecast_train['yhat'].values[-200:]
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"RMSE for the last 200 training points: {rmse}")

# Format the predicted 'ds' column to "2023-03-11T08:20:00+04:00 Dubai"
forecast_temp['ds'] = forecast_temp['ds'].dt.tz_localize('UTC').dt.tz_convert('Asia/Dubai').dt.strftime('%Y-%m-%dT%H:%M:%S%z') + ' Dubai'

# Save only the predicted 200 future values (ds and yhat columns) to a CSV file
output_path = r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\predictions.csv'
forecast_temp[['ds', 'yhat']].tail(200).to_csv(output_path, index=False)